# **Checking The DataBases**
아트테리어 작가와 작품 정보 DB 정리하기

In [1]:
from glob import glob
file_list = sorted(glob('../media/csv/*.*'))
file_list

['../media/csv/artist.csv',
 '../media/csv/artist_bak.csv',
 '../media/csv/artist_sns.csv',
 '../media/csv/assistant.csv',
 '../media/csv/gallery.csv',
 '../media/csv/store.csv',
 '../media/csv/store_bak.csv',
 '../media/csv/store_texts.csv']

## **1 Artist Data**
아트테리어 작가와 작품 정보 DB 정리하기

In [2]:
import pandas as pd
file_name = '../media/csv/store_texts.csv'
table_detail = pd.read_csv(file_name, header=None)
table_detail.head(2)

,0,1
0,A_01_1_banner_txt_01,산뜻한
1,A_01_1_banner_txt_02,영진떡집


In [3]:
file_name = '../media/csv/artist.csv'
columns_name = "name","group","number","title","subtitle","email","instagram","facebook","blog","photo"
table_raw = pd.read_csv(file_name, header=None)
table_raw.head(2)

,0,1,2,3,4
0,박현철,A,1,000-000-0000,email@email.com
1,나서윤,A,2,000-000-0000,email@email.com


## **2 Artist Name Check**
아트테리어 작가와 작품 정보 DB 정확도 확인하기

In [4]:
result = {}
result_raw = {}
import re
tokenizer = re.compile("[A-C]_[\d]{2}")
for _ in range(len(table_raw)):
    number = table_raw.iloc[_,2]
    text = f"{table_raw.iloc[_,1]}_{number:02d}"
    result_raw[text] = "".join(re.findall("[가-힣]+", table_raw.iloc[_,0]))

# Detail Table 에서 내용 추출하기
result = {"".join(tokenizer.findall(_)):table_detail[1][num].strip()  
          for num, _ in enumerate(table_detail[0])  
          if _.find('profile_txt_01') != -1}

# 2개의 결과값 비교하기
count_error = 0
for key, name in result_raw.items():
    name_check = result[key]
    if name != name_check:
        print(key)
        count_error += 1
if count_error == 0:
    print("=== Perfectly matched! ===")

=== Perfectly matched! ===


# **Building the Artist Table**
신규 정보를 보완하여 Artist Table 만들기

## **1 Reading the Tables**
작업에 필요한 정보들 한번 더 호출하기

In [5]:
import pandas as pd
file_name = '../media/csv/store_texts.csv'
table_detail = pd.read_csv(file_name, header=None)
table_detail.head(3)

,0,1
0,A_01_1_banner_txt_01,산뜻한
1,A_01_1_banner_txt_02,영진떡집
2,A_01_1_banner_txt_03,X 박현철 작가


In [6]:
file_name = '../media/csv/artist.csv'
table_raw = pd.read_csv(file_name, header=None).iloc[:,:3]
table_raw.columns = ["name","group","number"]
table_raw['number'] = [str(_)  for _ in table_raw['number']]
table_raw.head()

,name,group,number
0,박현철,A,1
1,나서윤,A,2
2,조유진,A,3
3,박은미,A,4
4,정수정,A,5


## **2 Extracted the Data**
- title, sub title

In [7]:
table_detail.iloc[7:12,:]

,0,1
7,A_01_1_image03_txt,사장님 적극적으로 컬러선정을 해주시고 그에 맞춰 해결방안을 모색하고 작업을 하였다.
8,A_01_1_profile_txt_01,박현철
9,A_01_1_profile_txt_02,시각예술가
10,A_01_1_profile_txt_03,"생명체에 대한 다양한 시점에서 관찰하고 연구하며, 설치작업을 하고 있다."
11,A_01_2_banner_txt_01,자주오고 싶은 카페


In [8]:
import re
def tokenizer_table(table, number):
    r"""Profile 정보를 추출하는 함수"""
    result = {}
    result_list = []
    regex_text = "[A-C]_[\d]{2}"
    check_text = "_1_profile_txt_0" + str(number)
    tokenizer = re.compile(regex_text + check_text)
    for _ in range(len(table)):
        name_index = table.iloc[_,0]
        name_text = table.iloc[_,1]
        check = tokenizer.findall(name_index)

        # 유효값이 발견시 작업
        if len(check) > 0:
            code = re.findall(regex_text, "".join(check))
            result["".join(code)] = name_text.strip()
            result_list.append(name_text.strip())
    return result_list # result
table_raw.insert(3,'title',tokenizer_table(table_detail, 2))
table_raw.insert(4,'subtitle',tokenizer_table(table_detail, 3))
for _col_name in ["email","instagram","facebook","blog","photo"]:
    table_raw[_col_name] = ''    
table_raw.tail(3)

,name,group,number,title,subtitle,email,instagram,facebook,blog,photo
34,안다혜,C,10,"설치미술, 타이포그라피 디자인",삶과 언어를 탐구하며 다양한 작업을 하고 있는 시각예술가 안다혜입니다.,,,,,
35,심효선,C,11,"평면, 설치",드로잉 작업을 바탕으로 평면과 설치작업을 하고 있습니다.,,,,,
36,임우현,C,12,"평면회화(동양화), 디지털페인팅, 드라마 소품작업 및 대역 등","동양화를 전공하여 주로 섬세하게 인물과 인형을 그리며 수작업, 디지털작업 모두 겸하...",,,,,


## **3 Extracted the SNS Data**
Social Network Data MA

In [9]:
file_name = '../media/csv/artist_sns.csv'
columns_name = "name","group","number","title","subtitle","email","instagram","facebook","blog","photo"
table_sns = pd.read_csv(file_name, header=0)
print(set(table_sns['sns_type'].to_list()))
table_sns.head(2)

{'instagram', 'email', 'facebook', 'blog'}


,artist,Index,sns_type,sns_url
0,박현철,A_01_1_profile_social_txt01,instagram,https://www.instagram.com/__parkhwa/
1,나서윤,A_02_1_profile_social_txt01,blog,https://www.instagram.com/s_t_u_d_i_o_n_a


In [10]:
regex_text  = "[A-C]_[\d]{2}"
result_list = []
for _ in range(len(table_sns)):
    table_instance = table_sns.iloc[_,:]
    name_index = "".join(re.findall(regex_text, table_instance['Index']))
    name_alphabet = "".join(re.findall('[A-C]+', name_index))
    name_number = str(int("".join(re.findall('[\d]+', name_index))))
    _index = table_raw[(table_raw['group']==name_alphabet) & (table_raw['number']==name_number)].index.to_list()[0]
    table_raw.loc[_index, table_instance['sns_type']] = table_instance['sns_url']
    
# filling the NaN data
import numpy as np
for _col in ["email","instagram","facebook","blog"]:
    data_list = []
    for _ in table_raw[_col]:
        if _ == '': data_list.append(np.nan) # NaN 데이터 추가하기
        else: data_list.append(_)            # 원본 데이터 그래도 덧붙이기
    table_raw[_col] = data_list
table_raw.head(3)

,name,group,number,title,subtitle,email,instagram,facebook,blog,photo
0,박현철,A,1,시각예술가,"생명체에 대한 다양한 시점에서 관찰하고 연구하며, 설치작업을 하고 있다.",NaN,https://www.instagram.com/__parkhwa/,NaN,NaN,
1,나서윤,A,2,"브랜딩디자인, 캘리그라피, 사진, 서양화","다양한 분야의 전문가, 예술가와 소통하며 풍요로운 예술 문화를 만들어가고 싶은 시각...",NaN,https://www.instagram.com/seoyoon_na/,NaN,https://www.instagram.com/s_t_u_d_i_o_n_a,
2,조유진,A,3,"일러스트레이터, UX/UI 디자이너","들숨에 선 하나 그리고, 날숨에 픽셀 하나 옮기고.",NaN,NaN,NaN,NaN,


## **4 Insert the Profile Photo**
프로필 사진 파일 연결하기
`\artist\A01.jpg`

In [11]:
table_raw["photo"] = [f"artist/{table_raw.iloc[_,1]}{int(table_raw.iloc[_,2]):02d}.jpg"   for _ in range(len(table_raw))]
table_raw.head(3)

,name,group,number,title,subtitle,email,instagram,facebook,blog,photo
0,박현철,A,1,시각예술가,"생명체에 대한 다양한 시점에서 관찰하고 연구하며, 설치작업을 하고 있다.",NaN,https://www.instagram.com/__parkhwa/,NaN,NaN,artist/A01.jpg
1,나서윤,A,2,"브랜딩디자인, 캘리그라피, 사진, 서양화","다양한 분야의 전문가, 예술가와 소통하며 풍요로운 예술 문화를 만들어가고 싶은 시각...",NaN,https://www.instagram.com/seoyoon_na/,NaN,https://www.instagram.com/s_t_u_d_i_o_n_a,artist/A02.jpg
2,조유진,A,3,"일러스트레이터, UX/UI 디자이너","들숨에 선 하나 그리고, 날숨에 픽셀 하나 옮기고.",NaN,NaN,NaN,NaN,artist/A03.jpg


# **Saving the file to CSV**
CSV 파일로 저장하기

In [12]:
table_raw.to_csv("test.csv", index=None, header=None)